In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.graphics.api import abline_plot
from random import sample
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc
from sklearn.linear_model import LogisticRegression

In [2]:
#Load the clean data
model_data= pd.read_csv("/Users/sa24/Desktop/Banking TLAB/Banking-Fraud-Analysis/Raw Banking Data/Modeling_df.csv")

In [3]:
# Store a list of features in x, isFraud won't be included because it's the target
X = model_data.drop(columns=['isFraud'])

# assign isFraud to y because it's the target
y= model_data['isFraud']

In [4]:
#split the data to run multiple trianing test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)